# Matrix Encryption
Makenna Worley and Nic Van Der Werf

An exploration into encrypting data through matrices transformations and modular arithmetic

In [32]:
# Importing and declaring global variables
import numpy as np
import math
import random

from ipywidgets import widgets
from IPython.display import display

MODULO = 59
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?'\n"
# print(len(ALPHABET))

KEY_MATRIX_2x2 = np.array([
    [3, 2],
    [5, 7]
])

text1 = open('text/one.txt').read()
text2 = open('text/two.txt').read()
text3 = open('text/our_super_secret_message.txt').read()

Functions that will help us with encrypting and decrypting our secret messages!

In [33]:
def generate_key_matrix(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))

        # Calculating the determinant with Mod
        a, b = key_matrix[0]
        c, d = key_matrix[1]
        det = (a*d - b*c) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")


def generate_key_matrix_nxn(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        det = int(round(np.linalg.det(key_matrix))) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            # All matrices must be invertible
            pass


def inverse_2x2_mod(matrix, modulo):
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    det_inv = pow(det, -1, modulo)
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo


# Inverting matrices with >2 rows and columns requires GCD since it is not guaranteed that rows and columns will be the same number, so we need to look at the coprime of them
def inverse_nxn_mod(matrix, modulo):
    A = np.array(matrix, dtype=int) % modulo
    n = A.shape[0]

    I = np.eye(n, dtype=int)

    # Perform Gaussian elimination
    for i in range(n):
        # Find pivot (row with non-zero element in column i)
        for r in range(i, n):
            if A[r, i] % modulo != 0:
                # Swap pivot row if necessary
                if r != i:
                    A[[i, r]] = A[[r, i]]
                    I[[i, r]] = I[[r, i]]
                break
        else:
            # No pivot found, matrix not invertible
            raise ValueError("Matrix not invertible modulo {}".format(modulo))

        # Normalize pivot row
        inv_pivot = pow(int(A[i, i]), -1, modulo)
        A[i] = (A[i] * inv_pivot) % modulo
        I[i] = (I[i] * inv_pivot) % modulo

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r, i] % modulo
                if factor != 0:
                    A[r] = (A[r] - factor * A[i]) % modulo
                    I[r] = (I[r] - factor * I[i]) % modulo

    return I % modulo


def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix


def encrypt_nxn(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    n = key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix


def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    cipher_matrix = np.array(numbers).reshape(-1, 2)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('*')
    return ''.join(plaintext_chars).rstrip('A')


def decrypt_nxn(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    dec = ''.join(plaintext_chars)
    return dec.rstrip('A')


def generate_random_string(alphabet, l):
    return ''.join(random.choice(alphabet) for _ in range(l))


def test_encryption_decryption(l):
    plaintext = generate_random_string(ALPHABET, l)
    ciphertext, ciphertext_matrix = encrypt(plaintext, KEY_MATRIX_2x2, MODULO, ALPHABET)
    decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
    return plaintext, decrypted_text, plaintext == decrypted_text

First testing our matrix to make sure it is invertible and therefore useable for our purposes.

In [34]:
inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)

for idx in range(1, 51):
    plaintext, decrypted_text, result = test_encryption_decryption(200)
    if result:
        print(f"Test {idx}: True")
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f"  Decrypted: {decrypted_text}")

Test 1: False
  Original: H.gMqvqhYg'olVqIqRbWLOGA
w.nfUercTEH,uPC'FtzyPPBSY bDulf.KVFzbdtpg'nQCJwpkAzgCRqiFBvQY'wkZtln
GMsARym XMPhpV
qZRAttppJGQturH wocXYv
.lhO'wjLNJyhEBG,HJdcdMhSpWE'TGffOAiJSm
oDmhFsMfAWRDwLjSIUbrMsMgQPV.A
  Decrypted: H.gMqvqhYg'olVqIqRbWLOGA
w.nfUercTEH,uPC'FtzyPPBSY bDulf.KVFzbdtpg'nQCJwpkAzgCRqiFBvQY'wkZtln
GMsARym XMPhpV
qZRAttppJGQturH wocXYv
.lhO'wjLNJyhEBG,HJdcdMhSpWE'TGffOAiJSm
oDmhFsMfAWRDwLjSIUbrMsMgQPV.
Test 2: True
Test 3: True
Test 4: True
Test 5: True
Test 6: True
Test 7: True
Test 8: True
Test 9: True
Test 10: True
Test 11: True
Test 12: True
Test 13: True
Test 14: True
Test 15: False
  Original: X ZPfccjLjZUmXC'mhlsSozIOqH!FJ,.Sy?,x'ZiGphhINN'KgOniVRwHsk,sFUoIbsUE
JxEv
wpAYdk!XrEz'?nmAVffj,RRLejwG!IGJqT!cnA.eWo!KsF.q?PD'bfqRlmmkEOf P,tKBUTOlIHqIex ORhTsBxJQPryhLInrEzJngnEftyOELBNe,qAh'JP!fqeA
  Decrypted: X ZPfccjLjZUmXC'mhlsSozIOqH!FJ,.Sy?,x'ZiGphhINN'KgOniVRwHsk,sFUoIbsUE
JxEv
wpAYdk!XrEz'?nmAVffj,RRLejwG!IGJqT!cnA.eWo!KsF.q?PD'bfqRlmmkEOf P,tKBU

Now let's look at our encrypting a message using our predefined KEY_MATRIX_2x2

In [38]:
MESSAGE = text1

inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)
print(f"Key Matrix:\n{KEY_MATRIX_2x2}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, KEY_MATRIX_2x2, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")
print(f"Condition: {np.linalg.cond(KEY_MATRIX_2x2)}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)


Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[ 6 32]
 [21 11]]
Ciphertext Matrix:
[[ 4 38]
 [58 38]
 [33 36]
 [46 55]
 [34  4]
 [16  6]
 [12 12]
 [45  4]
 [ 2 10]
 [ 4  6]
 [30 44]
 [54 23]
 [16  4]
 [37 29]
 [ 4  6]
 [31 29]
 [38 44]
 [ 1 45]
 [13 34]
 [ 0  9]
 [51 30]
 [10 53]
 [40 45]
 [17 42]
 [46  7]
 [47  0]
 [15 17]
 [ 0  9]
 [44 28]
 [10 53]
 [50 38]
 [55 22]
 [56 48]
 [ 3 30]
 [52 39]
 [11 10]
 [ 4 43]
 [38 18]
 [30 48]
 [29 29]
 [42 19]
 [ 5 22]
 [29  6]
 [44 28]
 [11 35]
 [56 46]
 [14 43]]
Ciphertext: Em
mhku!iEQGMMtECKEGes,XQEldEGfdmsBtNiAJzeK.otRquHvAPRAJscK.ym!W?wDe nLKErmSewddqTFWdGscLj?uOr
Condition: 7.780565547548909


Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()

In [39]:
MESSAGE = text2

key_matrix, inverse_key_matrix = generate_key_matrix(2, MODULO)

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, key_matrix, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")
print(f"Condition: {np.linalg.cond(key_matrix)}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Key Matrix:
[[41 23]
 [58 22]]
Inverse Key Matrix:
[[33 54]
 [31 32]]
Ciphertext Matrix:
[[ 6 38]
 [45 26]
 [43  2]
 [21 23]
 [ 9 39]
 [12 32]
 [ 6 58]
 [49 57]
 [47 28]
 [49 15]
 [ 4  8]
 [21 23]
 [40 53]
 [37 37]
 [ 0 25]
 [ 6 21]
 [12 49]
 [ 0 29]
 [43 28]
 [24  9]
 [26 43]
 [16 30]
 [14 36]
 [10 45]
 [50 41]
 [34 27]
 [ 7  2]
 [26 35]
 [36 20]
 [ 7 52]
 [53  0]
 [ 7 24]
 [30 13]
 [36 31]
 [13  4]]
Ciphertext: GmtarCVXJnMgG
x'vcxPEIVXo.llAZGVMxAdrcYJarQeOkKtypibHCajkUH .AHYeNkfNE
Condition: 13.951471119148144


Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()

In [40]:
MESSAGE = text3
KEY_MATRIX_SIZE = 7

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Plain Text Length: 93
Ciphertext Length: 1722
Key Matrix:
[[17 11 53  7 28 39 16]
 [37 21 14 42 33 57 10]
 [30 36 13 29 42 40 52]
 [39 54 44 47  5 20  0]
 [51 29  1 28 26  1  1]
 [52 45 25  3 56 34  8]
 [10 32 40 31 30 24 19]]
Inverse Key Matrix:
[[40  1 49 25 50 42  7]
 [13 18 22  7 46 33 49]
 [ 7  9 25 50 14 23 13]
 [31 51 13 35 28 19 20]
 [51 40 49 25 20 30 18]
 [16 24 57 23 13 36  7]
 [27 53 40  5 37 14  6]]
Ciphertext: ZOK!.UMPydJR!b!kbXWPjcktqrgQDBszxubJOlDRCG!c itzHjbP,Qk ar BK flhJ.'sMjTSGIzShvHERedQhnY'sUCdHWnplxXWbyJ
rxaDqHDvPZjwIuf'IToowfenOHzDuGaSOO lcuk!l PubegXCjLve,rELdntmFG'UCwjNo?VrsZniZQ.DQeRxyQetuQerwQuPSZqcKKaYrfYsWMrTjXFK,ZBNEfwO
e.'EFlOQxMqbOjgveF!tmZqcKKaYqXGshpQmEpvhswNQB?sllRAaOXYpwBdaNBue.'EFlOZQ.DQeRlMoUPyeZqcKKaY
h
sixjkJEK,ngoVWb
jbe.'EFlORGz,yndHNUlU?UretzQsS.ZB
cDTBMJttezKz
vQ.ggJT?zXyk'c whwL VpIiV
KLLxUXEI,kPYRT
'XuoT zCBwajR!tLDHHu!ZfqTnGURVGQqIRfJ wUGsIFBlxFgTLRxslOxzB'WEI,kPYRBJHM
q
rduS ChmQHN
l'IcCXeX,DiqKaKexZmt!KX.gY
cWJEsuDDmzq,rafq.KiuHfuQCTJmw

Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()